Choosing the Database Type

There are several types of databases we can consider for this project, including:

MySQL

Pros: Open-source, widely used, strong community support, ACID compliance, good for complex queries and joins.
Cons: Not as flexible for unstructured data, can be slower for very large datasets compared to some NoSQL options.

PostgreSQL

Pros: Open-source, highly extensible, supports advanced data types, strong community support, ACID compliance.
Cons: Slightly more complex to set up and manage compared to MySQL, can have a steeper learning curve.

SQLite

Pros: Lightweight, easy to set up, self-contained, good for small to medium-sized applications.
Cons: Limited concurrency support, not suitable for very large datasets or high-traffic applications.

MongoDB

Pros: NoSQL, schema-less, great for unstructured data, flexible, good for hierarchical data.
Cons: Not ACID compliant by default (though transactions are supported in recent versions), can be less efficient for complex queries and joins.

Ironhack initiated us with mysql, so by practicality mysql was chosen.

In [23]:
import pandas as pd
import mysql.connector
from mysql.connector import Error
from getpass import getpass

def create_connection():
    """Create a database connection and return the connection object."""
    conn = None
    try:
        # Prompt for password securely
        password = getpass(prompt='Enter your MySQL password: ')

        # Connect to MySQL server and specify the database 'yummy'
        conn = mysql.connector.connect(
            host='127.0.0.1',  # Your MySQL server host
            port='3306',       # Your MySQL server port
            user='root',       # Your MySQL username
            password=password, # Securely entered password
            database='yummy'   # Specify the database 'yummy'
        )

        if conn.is_connected():
            print('Connected to MySQL server and using database yummy')
        return conn

    except Error as e:
        print("Error while connecting to MySQL", e)
        return None



In [20]:
import pandas as pd
import mysql.connector
from mysql.connector import Error

def create_connection():
    """Create a database connection."""
    try:
        conn = mysql.connector.connect(
            host='localhost',
            user='root',
            password='password',
            database='yummy'
        )
        if conn.is_connected():
            print("Connected to MySQL server and using database yummy")
            return conn
    except Error as e:
        print(f"Error: {e}")
        return None

def create_tables(conn):
    """Create tables in the database."""
    try:
        cursor = conn.cursor()
        cursor.execute("""
        CREATE TABLE IF NOT EXISTS recipes (
            recipeid INT PRIMARY KEY,
            name VARCHAR(255),
            cooktime INT,
            preptime INT,
            totaltime INT,
            recipecategory VARCHAR(255),
            recipeingredientparts TEXT,
            aggregatedrating FLOAT,
            reviewcount INT,
            recipeinstructions TEXT
        )""")

        cursor.execute("""
        CREATE TABLE IF NOT EXISTS reviews (
            reviewid INT PRIMARY KEY,
            recipeid INT,
            rating FLOAT,
            FOREIGN KEY (recipeid) REFERENCES recipes(recipeid)
        )""")

        cursor.execute("""
        CREATE TABLE IF NOT EXISTS vegetables (
            vegetableid INT PRIMARY KEY,
            name VARCHAR(255),
            month VARCHAR(50),
            country VARCHAR(100)
        )""")

        cursor.execute("""
        CREATE TABLE IF NOT EXISTS fruits (
            fruitid INT PRIMARY KEY,
            name VARCHAR(255),
            month VARCHAR(50),
            country VARCHAR(100)
        )""")

        conn.commit()
        print("Tables created successfully")

    except Error as e:
        print(f"Error while creating tables: {e}")

def insert_data(conn, table, data):
    """Insert data into the specified table."""
    try:
        cursor = conn.cursor()
        cols = ",".join([str(i) for i in data.columns.tolist()])
        placeholders = ",".join(["%s"] * len(data.columns))

        for i, row in data.iterrows():
            sql = f"INSERT INTO {table} ({cols}) VALUES ({placeholders})"
            cursor.execute(sql, tuple(row))

        conn.commit()
        print(f"Data inserted successfully into {table} table")

    except Error as e:
        print(f"Error while inserting data into {table}: {e}")

def main():
    conn = create_connection()

    if conn:
        create_tables(conn)

        # Read data from CSV files
        df_recipes = pd.read_csv('recipes_filtered.csv')
        df_reviews = pd.read_csv('modif_reviews.csv')
        df_vegetables = pd.read_csv('vegetable_data.csv')
        df_fruits = pd.read_csv('fruit_data.csv')

        # Insert data into tables
        insert_data(conn, 'recipes', df_recipes)
        insert_data(conn, 'reviews', df_reviews)
        insert_data(conn, 'vegetables', df_vegetables)
        insert_data(conn, 'fruits', df_fruits)

        # Close the connection
        conn.close()

if __name__ == "__main__":
    main()


Error: 1045 (28000): Access denied for user 'root'@'localhost' (using password: YES)


I process the Querries on MySql workbench